In [2]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path = [repo_dir] + sys.path



/Users/marijnkoolen/Code/Huygens/republic-project


In [3]:
from republic.elastic.republic_elasticsearch import initialize_es


rep_es = initialize_es(host_type="external", timeout=60)



In [29]:
%%time

import gzip
import json

def make_resolution_size_query(size_min: int = 0, size_max: int = 100000):
    return {
        'function_score': {
            'query': {
                'bool': {
                    'must': [
                        {'match': {'type': 'resolution'}},
                        {'range': {'metadata.session_year': {'gte': 1700, 'lte': 1797}}},
                        {'range': {'stats.words': {'lte': size_max, 'gte': size_min}}}
                    ]
                }            
            },
            'random_score': {
                'seed': seed,
                'field': '_seq_no'
            }
        }
    }
# query = {
#     "match": {
#         "metadata.session_year": 1672
#     }
# }

seed = 937498

res_lengths = [
    {'length': 'short', 'size_min': 0, 'size_max': 50},
    {'length': 'mid', 'size_min': 100, 'size_max': 300},
    {'length': 'long', 'size_min': 500, 'size_max': 100000}
]

for res_length in res_lengths:
    query = make_resolution_size_query(size_min=res_length['size_min'], size_max=res_length['size_max'])
    resolutions = rep_es.retrieve_resolutions_by_query(query, size=1000)
    words = [res.stats['words'] for res in resolutions]
    print(f"{res_length['length']: <8}{len(words): >5}\t{min(words): >5}\t{max(words): >8}")
    out_file = f"../../data/resolutions/resolutions-{res_length['length']}.json.gz"
    with gzip.open(out_file, 'wt') as fh:
        json.dump([res.json for res in resolutions], fh)

#     for res in resolutions:
#         for para in res.paragraphs:
#             print(para.text)

short    1000	   22	      50
mid      1000	  100	     300
long     1000	  500	   19800
CPU times: user 21.4 s, sys: 784 ms, total: 22.2 s
Wall time: 27.4 s


In [31]:
query = {
    'bool': {
        'must': [
            {'match': {'type': 'resolution'}},
            {'range': {'metadata.session_year': {'gte': 1672, 'lte': 1672}}},
        ]
    }                
}

resolutions = rep_es.retrieve_resolutions_by_query(query, size=10)
[res.id for res in resolutions]

[]

In [7]:
resolutions = [hit["_source"] for hit in response["hits"]["hits"]]
len(resolutions)

2605

In [9]:
import json

res_file = "../../data/resolutions/rampjaar-ordinars-resolutions.json"

with open(res_file, 'wt') as fh:
    json.dump(resolutions, fh)